Подключение библиотек. YOLO (You Only Look Once) - это алгоритм машинного обучения, который используется для обнаружения объектов на изображениях и видео.

In [ ]:
from ultralytics import YOLO
import numpy as np
import cv2

Загрузка модели и детекция

In [ ]:
# Загружаем модель YOLOv8x
model = YOLO("yolov8x.pt")

# Загружаем изображение
original_image = cv2.imread("test_img.jpg")

# Выполняем детекцию
results = model.predict(original_image, classes=[0], conf=0.2, iou=0.7)
# classes=[0] - распознавание объектов с классом person

Задание 1.

Необходимо распознать всех людей на изображении, отобразить их описывающим прямоугольником.

Посчитать их количество, отобразить на изображении полученное значение.

In [ ]:
image = original_image.copy()
people = []

for detection in results[0].boxes.xyxy:  
        x1, y1, x2, y2 = detection.cpu().numpy().astype(int) 
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2
        people.append([x1, y1, x2, y2, center_x, center_y])   
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

cv2.putText(image, f'People: {len(results[0])}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 2)

In [ ]:
cv2.imshow("Результат", image)
cv2.waitKey(0)

Задание 2

Определить наличие группы людей и отобразить её описывающим прямоугольником со значением количества людей в группе.

Группа - люди, расположенные на расстоянии менее 100 пикселей между центрами описывающих рамок

Определяем людей по группам

In [ ]:
groups = []

for person in people:
    
    # Проверяем, есть ли группа, в которую можно добавить человека
    found_group = False
    for group in groups:
        for person_in_group in group:
            # Расстояние между центрами
            distance = np.sqrt((person[4] - person_in_group[4])**2 + (person[5] - person_in_group[5])**2)
            # Если расстояние меньше 100 пикселей, добавляем человека к группе
            if distance < 100:
                group.append(person)
                found_group = True
                break
            if found_group:
                break
       
    # Если группа не найдена, создаем новую группу
    if not found_group:
        groups.append([person])     

Добавляем прямоугольники и надписи

In [ ]:
image = original_image.copy()

for group in groups:
    if len(group) > 1:
        x1, y1 = 10**10, 10**10
        x2, y2 = 0, 0
        for person in group:
            cur_x1, cur_y1, cur_x2, cur_y2 = person[:4]
            if cur_x1 < x1:
                x1 = cur_x1
            if cur_y1 < y1:
                y1 = cur_y1
            if cur_x2 > x2:
                x2 = cur_x2
            if cur_y2 > y2:
                y2 = cur_y2
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2) 
        cv2.putText(image, f'People: {len(group)}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255, 0), 2)

In [ ]:
cv2.imshow("Результат", image)
cv2.waitKey(0)